In [1]:
import pandas as pd
import sqlite3
from pandasql import sqldf

In [2]:
#Added all data as dataframes
movie_gross = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
name_basics = pd.read_csv('zippedData/imdb.name.basics.csv.gz')
title_akas = pd.read_csv('zippedData/imdb.title.akas.csv.gz')
title_basics= pd.read_csv('zippedData/imdb.title.basics.csv.gz')
title_crew = pd.read_csv('zippedData/imdb.title.crew.csv.gz')
title_principals = pd.read_csv('zippedData/imdb.title.principals.csv.gz')
title_ratings = pd.read_csv('zippedData/imdb.title.ratings.csv.gz')
movie_info = pd.read_csv('zippedData/rt.movie_info.tsv.gz', '\t')
reviews = pd.read_csv('zippedData/rt.reviews.tsv.gz', '\t', encoding='windows-1252')
movies = pd.read_csv('zippedData/tmdb.movies.csv.gz')
movie_budgets = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

In [3]:
movie_budgets['production_budget'] = movie_budgets['production_budget'].map(lambda x: x.strip('$'))
movie_budgets['production_budget'] = movie_budgets['production_budget'].map(lambda x: x.replace(',', ''))
movie_budgets['production_budget'] = movie_budgets['production_budget'].astype('int64')
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   int64 
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(2), object(4)
memory usage: 271.2+ KB


In [4]:
movie_budgets['domestic_gross'] = movie_budgets['domestic_gross'].map(lambda x: x.strip('$'))
movie_budgets['domestic_gross'] = movie_budgets['domestic_gross'].map(lambda x: x.replace(',', ''))
movie_budgets['domestic_gross'] = movie_budgets['domestic_gross'].astype('int64')
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   int64 
 4   domestic_gross     5782 non-null   int64 
 5   worldwide_gross    5782 non-null   object
dtypes: int64(3), object(3)
memory usage: 271.2+ KB


In [5]:
movie_budgets['worldwide_gross'] = movie_budgets['worldwide_gross'].map(lambda x: x.strip('$'))
movie_budgets['worldwide_gross'] = movie_budgets['worldwide_gross'].map(lambda x: x.replace(',', ''))
movie_budgets['worldwide_gross'] = movie_budgets['worldwide_gross'].astype('int64')
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   int64 
 4   domestic_gross     5782 non-null   int64 
 5   worldwide_gross    5782 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 271.2+ KB


In [6]:
movie_budgets['release_month'] = movie_budgets['release_date'].apply(lambda x: x[:3])

In [7]:
movie_budgets['profit'] = movie_budgets['worldwide_gross'] - movie_budgets['production_budget']
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_month,profit
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,Dec,2351345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,May,635063875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,Jun,-200237650
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,May,1072413963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,Dec,999721747


In [8]:
pysqldf = lambda q: sqldf(q, globals())

In [9]:
table = pysqldf('''select original_title, primary_title, primary_name, production_budget, worldwide_gross, genres, category, runtime_minutes, averagerating, numvotes, profit, release_month
                    from movie_budgets mb
                    join title_basics tb
                    on mb.movie == tb.primary_title
                    join title_principals
                    using(tconst)
                    join title_ratings
                    using (tconst)
                    join name_basics
                    using(nconst)''')

In [10]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27145 entries, 0 to 27144
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   original_title     27145 non-null  object 
 1   primary_title      27145 non-null  object 
 2   primary_name       27145 non-null  object 
 3   production_budget  27145 non-null  int64  
 4   worldwide_gross    27145 non-null  int64  
 5   genres             27085 non-null  object 
 6   category           27145 non-null  object 
 7   runtime_minutes    26168 non-null  float64
 8   averagerating      27145 non-null  float64
 9   numvotes           27145 non-null  int64  
 10  profit             27145 non-null  int64  
 11  release_month      27145 non-null  object 
dtypes: float64(2), int64(4), object(6)
memory usage: 2.5+ MB


In [11]:
table.head()

,original_title,primary_title,primary_name,production_budget,worldwide_gross,genres,category,runtime_minutes,averagerating,numvotes,profit,release_month
0,Abatâ,Avatar,Ai Hashimoto,425000000,2776345279,Horror,actress,93.0,6.1,43,2351345279,Dec
1,Abatâ,Avatar,Yûsuke Yamada,425000000,2776345279,Horror,writer,93.0,6.1,43,2351345279,Dec
2,Abatâ,Avatar,Rikako Sakata,425000000,2776345279,Horror,actress,93.0,6.1,43,2351345279,Dec
3,Abatâ,Avatar,Nako Mizusawa,425000000,2776345279,Horror,actress,93.0,6.1,43,2351345279,Dec
4,Abatâ,Avatar,Hideki Ikari,425000000,2776345279,Horror,composer,93.0,6.1,43,2351345279,Dec


In [12]:
table.groupby(['original_title']).describe()

production_budget                               \
                                       count        mean  std         min   
original_title                                                              
#Horror                                 10.0   1500000.0  0.0   1500000.0   
10 Cloverfield Lane                     10.0   5000000.0  0.0   5000000.0   
10 Days in a Madhouse                   10.0  12000000.0  0.0  12000000.0   
12 Rounds                                9.0  20000000.0  0.0  20000000.0   
12 Strong                               10.0  35000000.0  0.0  35000000.0   
...                                      ...         ...  ...         ...   
Zázrak                                  10.0  28000000.0  0.0  28000000.0   
xXx: Return of Xander Cage              10.0  85000000.0  0.0  85000000.0   
À Queima Roupa                           6.0   3000000.0  0.0   3000000.0   
À bout portant                          10.0   3000000.0  0.0   3000000.0   
Évolution                               10.0  80000000.0  0.0  80000000.0   

                                                                            \
                                   25%         50%         75%         max   
original_title                                                               
#Horror                      1500000.0   1500000.0   1500000.0   1500000.0   
10 Cloverfield Lane          5000000.0   5000000.0   5000000.0   5000000.0   
10 Days in a Madhouse       12000000.0  12000000.0  12000000.0  12000000.0   
12 Rounds                   20000000.0  20000000.0  20000000.0  20000000.0   
12 Strong                   35000000.0  35000000.0  35000000.0  35000000.0   
...                                ...         ...         ...         ...   
Zázrak                      28000000.0  28000000.0  28000000.0  28000000.0   
xXx: Return of Xander Cage  85000000.0  85000000.0  85000000.0  85000000.0   
À Queima Roupa               3000000.0   3000000.0   3000000.0   3000000.0   
À bout portant               3000000.0   3000000.0   3000000.0   3000000.0   
Évolution                   80000000.0  80000000.0  80000000.0  80000000.0   

                           worldwide_gross               ...  numvotes  \
                                     count         mean  ...       75%   
original_title                                           ...             
#Horror                               10.0          0.0  ...    3092.0   
10 Cloverfield Lane                   10.0  108286422.0  ...  260383.0   
10 Days in a Madhouse                 10.0      14616.0  ...    1114.0   
12 Rounds                              9.0   17306648.0  ...      11.0   
12 Strong                             10.0   71118378.0  ...   50155.0   
...                                    ...          ...  ...       ...   
Zázrak                                10.0   64474705.0  ...     114.0   
xXx: Return of Xander Cage            10.0  345033359.0  ...   77976.0   
À Queima Roupa                         6.0          0.0  ...      15.0   
À bout portant                        10.0          0.0  ...   12706.0   
Évolution                             10.0   98341932.0  ...    5266.0   

                                     profit                                 \
                                 max  count         mean  std          min   
original_title                                                               
#Horror                       3092.0   10.0   -1500000.0  0.0   -1500000.0   
10 Cloverfield Lane         260383.0   10.0  103286422.0  0.0  103286422.0   
10 Days in a Madhouse         1114.0   10.0  -11985384.0  0.0  -11985384.0   
12 Rounds                       11.0    9.0   -2693352.0  0.0   -2693352.0   
12 Strong                    50155.0   10.0   36118378.0  0.0   36118378.0   
...                              ...    ...          ...  ...          ...   
Zázrak                         114.0   10.0   36474705.0  0.0   36474705.0   
xXx: Return of Xander Cage   77976.0  